In [1]:
#normal python stuff
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os

In [2]:
#import local helper
import helpers

In [3]:
#local cadcad stuff

os.chdir("/Users/zargham/Documents/GitHub/cadcad-ri")
from cadcad.spaces import space
from cadcad.dynamics import block
from cadcad.spaces import EmptySpace, Bit, Real
from cadcad.points import Point
from cadcad.systems import Experiment

In [4]:
print(EmptySpace)

Empty space EmptySpace


In [6]:
spaceType = type(EmptySpace)

In [7]:
print(spaceType)

<class 'cadcad.spaces.Space'>


In [8]:
## let's try something weird
## we will make a space containing a space

@space
class ContributionSchema:
    schema: spaceType

In [9]:
print(ContributionSchema)

Space ContributionSchema has dimensions {'schema': 'Space'}


In [10]:
@space
class InitSchema:
    a: int
    b: float
    c: str

In [11]:
myPt = Point(ContributionSchema, {'schema':InitSchema })

In [12]:
myPt

In [14]:
print(myPt)

Point in space ContributionSchema has data
{
    "schema": "Space InitSchema has dimensions {'a': 'int', 'b': 'float', 'c': 'str'}"
}



In [15]:
@block
def randomSchemaChange(domain:Point[ContributionSchema])->Point[ContributionSchema]:
    input = deepcopy(domain)
    rng = np.random.randn()

    if rng <-1.0:
        #drop a dimension
        options = list(input.data['schema'].dimensions().keys())
        print(options)
        dk = np.random.choice(options)
        NewSchema = helpers.drop_dimension(input.data['schema'], dk)
        
        return Point(ContributionSchema, {'schema':NewSchema})

    elif rng >-1.0:
        #add a dimension
        nk = np.random.choice(helpers.WORDS)
        NewSchema = Bit.rename_dims({'bit':nk}) + input.data['schema']

        return Point(ContributionSchema, {'schema':NewSchema})
    
    else:
        return input
    #else:
        #nest a dimension


In [16]:
CS2 = ContributionSchema**2

In [17]:
print(CS2)

Space 2-ContributionSchema has dimensions {'contributionschema_0': 'ContributionSchema', 'contributionschema_1': 'ContributionSchema'}


In [18]:
print(Real)

Space Real has dimensions {'real': 'float'}


In [19]:
CartesianPlane = Real * Real

In [21]:
print(CartesianPlane)

Space Real*Real has dimensions {'real_0': 'Real', 'real_1': 'Real'}


In [22]:
CartesianPlane = CartesianPlane.rename_dims({'real_0':'x', 'real_1':'y'})

In [23]:
print(CartesianPlane)

Space Real*Real has dimensions {'x': 'Real', 'y': 'Real'}


In [24]:
@space
class OtherCartesianPlane:
    x:float
    y:float

In [25]:
print(OtherCartesianPlane)

Space OtherCartesianPlane has dimensions {'x': 'float', 'y': 'float'}


In [26]:
@block
def schemaDistance(domain:Point[CS2])->Point[Real]:
    schema0 = domain['contributionschema_0'].data['schema'].unroll_schema()
    schema1 = domain['contributionschema_1'].data['schema'].unroll_schema()

    #schema0 and schema1 are nested dictionaries
    #include a distance measure for nested dictionaries here
    value = 0.0

    return Point(Real, value)


In [27]:
schemaPt = Point(ContributionSchema, {'schema':InitSchema})

In [28]:
print(schemaPt)

Point in space ContributionSchema has data
{
    "schema": "Space InitSchema has dimensions {'a': 'int', 'b': 'float', 'c': 'str'}"
}



In [29]:
print(randomSchemaChange(schemaPt))

Point in space ContributionSchema has data
{
    "schema": "Space Bit+InitSchema has dimensions {'apples': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}



In [30]:
print(randomSchemaChange(randomSchemaChange(schemaPt)))

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+InitSchema has dimensions {'apples': 'bool', 'bananas': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}



In [31]:
experiment_params = {"iteration_n": 2, "steps": 10}
my_experiment = Experiment(schemaPt, experiment_params, (randomSchemaChange,))

results = my_experiment.run()

['ho', 'ho_1', 'bananas', 'apples', 'byebye', 'ho_1_1', 'apples_1', 'a', 'b', 'c']


In [32]:
results

In [19]:
results[0].data

In [33]:
print(results[1].data[0])

Point in space ContributionSchema has data
{
    "schema": "Space Bit+InitSchema has dimensions {'apples': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}



In [34]:
for r in results:
    for pt in r.data:
        print(pt)

Point in space ContributionSchema has data
{
    "schema": "Space Bit+InitSchema has dimensions {'apples': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+InitSchema has dimensions {'ho': 'bool', 'apples': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+Bit+InitSchema has dimensions {'byebye': 'bool', 'ho': 'bool', 'apples': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+Bit+Bit+InitSchema has dimensions {'apples': 'bool', 'byebye': 'bool', 'ho': 'bool', 'apples_1': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}

Point in space ContributionSchema has data
{
    "schema": "Space Bit+Bit+Bit+Bit+Bit+InitSchema has dimensions {'bananas': 'bool', 'apples': 'bool', 'byebye': 'bool', 'ho': 'bool', 'apples_1': 'bool', 'a': 'int', 'b': 'float', 'c': 'str'}"
}

Point 

scratchwork from session with Govrn research team below

is not expected to run

In [ ]:
SpaceType = type(EmptySpace)

BlockType = type(someblock)

@space
class DistanceMetric:
    metric: BlockType
    space: SpaceType

    # constrain on metric domain
    pass


@space
class GovrnUser:
    preferedDistanceMetric: DistanceMetric
    prederedSchema: ContributionSchema

@space
class GovrnUniverse:
    userSchemas: list[GovrnUser]
    canonicalSchema: ContributionSchema

@block
def schemaAverage(domain:Point[GovrnUniverse])-> Point[GovrnUniverse]:
    ## 
    pass
